In [3]:
!pip install torch
!pip install torchvision
!pip install torchaudio
!pip install tokenizers
!pip install transformers


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from transformers import pipeline
import numpy as np

# Initialize sentiment analyzer with specific model to avoid downloading issues
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

print("Sentiment analysis model loaded successfully!")

Sentiment analysis model loaded successfully!


In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 1. Load the tokenizer & model (pinning model + rev)
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
revision   = "af0f99b"

tokenizer = AutoTokenizer.from_pretrained(model_name, revision=revision)
model     = AutoModelForSequenceClassification.from_pretrained(model_name, revision=revision)
model.eval()  # disable dropout

texts = [
    "I love this product! It's amazing.",
    "The service was terrible and I'm very disappointed.",
    "It's okay, not great but not bad either."
]

# 2. Loop through your texts
for txt in texts:
    # tokenize & forward
    inputs = tokenizer(txt, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits

    # convert logits → probabilities
    probs = torch.softmax(logits, dim=-1)[0]
    # pick the highest
    score = probs.max().item()
    label = model.config.id2label[probs.argmax().item()]

    print(f"{txt!r} → {label} ({score:.2f})")

"I love this product! It's amazing." → POSITIVE (1.00)
"The service was terrible and I'm very disappointed." → NEGATIVE (1.00)
"It's okay, not great but not bad either." → POSITIVE (1.00)


In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 1) Load & pin the exact model
MODEL   = "distilbert-base-uncased-finetuned-sst-2-english"
REV     = "af0f99b"
tokenizer = AutoTokenizer.from_pretrained(MODEL, revision=REV)
model     = AutoModelForSequenceClassification.from_pretrained(MODEL, revision=REV)
model.eval()

# 2) Prompt the user
custom_text = input("Enter a sentence for sentiment analysis: ")

# 3) Tokenize & run (no gradients)
inputs = tokenizer(custom_text, return_tensors="pt", truncation=True)
with torch.no_grad():
    logits = model(**inputs).logits

# 4) Convert to probabilities & pick the top label
probs = torch.softmax(logits, dim=-1)[0]
score, idx = probs.max(dim=0)
label = model.config.id2label[idx.item()]

# 5) Display results
print(f"\nSentiment:   {label}")
print(f"Confidence:  {score.item():.2f}")



Sentiment:   NEGATIVE
Confidence:  1.00


In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 1) Load & pin your model + tokenizer
MODEL   = "distilbert-base-uncased-finetuned-sst-2-english"
REV     = "af0f99b"
tokenizer = AutoTokenizer.from_pretrained(MODEL, revision=REV)
model     = AutoModelForSequenceClassification.from_pretrained(MODEL, revision=REV)
model.eval()

# 2) Your “long” text
long_text = """
The product is good overall, but there are some issues with battery life.
I wish it lasted longer. However, the design is sleek, and I’m happy with the performance so far.
"""

# 3) Tokenize with truncation (and padding, if you want a batch later)
inputs = tokenizer(
    long_text,
    return_tensors="pt",
    truncation=True,      # chops off beyond 512 tokens
    padding=False,        # no pad needed for single example
    max_length=512
)

# 4) Forward + no grad
with torch.no_grad():
    logits = model(**inputs).logits  # shape (1, 2)

# 5) Softmax → probabilities
probs = torch.softmax(logits, dim=-1)[0]  # tensor([neg_score, pos_score])

# 6) Print out both labels
for idx, score in enumerate(probs):
    label = model.config.id2label[idx]
    print(f"{label:>8}: {score.item():.2f}")


NEGATIVE: 0.00
POSITIVE: 1.00
